SciDB is a scale-out DBMS designed for scientific use cases, with built-in capabilities for sophisticated mathematical calculations beyond the typical domain of SQL/NoSQL systems. This workbook demonstrates using SciDB's R interface for scalable query and analysis of 1000 Genomes phase 3 genotype data, including simple aggregations such as computing transition/transversion ratios, and more advanced calculations such as principal component analysis and estimating linkage disequilibrium.

First, we connect to SciDB and obtain handles to the data arrays. For development/testing on a modest machine, we'll just look at chromosomes 21 and 22 across the 2,504 individuals; but SciDB can scale out on a cluster to handle arbitrarily large datasets.

In [ ]:
#require(ggplot2)
#require(pipeR)
#require(scidb)
import matplotlib.pyplot as plt
from scidbpy import connect

#scidbconnect()
sdb = connect()

In [11]:
#SAMPLE <- scidb("KG_SAMPLE")
SAMPLE = sdb.wrap_array('KG_SAMPLE')

#CHROMOSOME <- scidb("KG_CHROMOSOME")
CHROMOSOME = sdb.wrap_array('KG_CHROMOSOME')

#VARIANT <- merge(scidb("KG_VARIANT"), subset(CHROMOSOME, "chromosome='10' or chromosome='21' or chromosome='22'"))
VARIANT = sdb.merge(sdb.wrap_array('KG_VARIANT'), 
                    sdb.afl.filter("KG_CHROMOSOME", "chromosome='10' or chromosome='20' or chromosome='21'"))

#VARIANT <- scidbeval(VARIANT)
VARIANT.eval()

#GENOTYPE <- project(merge(scidb("KG_GENOTYPE"),project(VARIANT,'chromosome')),
#                    c("allele_1","allele_2","phase"))
GENOTYPE = sdb.afl.project(
    sdb.merge(
        sdb.wrap_array('KG_GENOTYPE'),
        sdb.afl.project(VARIANT, 'chromosome')
        ),
    'allele_1', 'allele_2', 'phase'
    )

The `VARIANT` array for chomosomes 21 and 22 has been stored in SciDB's memory (but not the memory of our local R process) by the `scidbeval()` expression above. In contrast, the `GENOTYPE` array is merely a lazy expression, denoting but not materializing the subset of all the genotypes on chromosomes 21 and 22. Because `GENOTYPE` is our biggest array by far, we avoid copying any significant portion of it.

Let's take a look at the schema.

In [51]:
#head(VARIANT)
VARIANT.todataframe().head()

reference alternate           id  qual  \
chromosome_id start end   alternate_id                                          
9             60494 60494 1                    A         G            .   100   
              60515 60515 1                    C         T            .   100   
              60523 60523 1                    T         G  rs148087467   100   
              60607 60607 1                    G         A            .   100   
              60684 60684 1                    A         C            .   100   

                                       filter  \
chromosome_id start end   alternate_id          
9             60494 60494 1              PASS   
              60515 60515 1              PASS   
              60523 60523 1              PASS   
              60607 60607 1              PASS   
              60684 60684 1              PASS   

                                                                       info  \
chromosome_id start end   alternate_id                                        
9             60494 60494 1             AC=27;AF=0.00539137;AN=5008;NS=2504   
              60515 60515 1             AC=1;AF=0.000199681;AN=5008;NS=2504   
              60523 60523 1              AC=91;AF=0.0181709;AN=5008;NS=2504   
              60607 60607 1             AC=1;AF=0.000199681;AN=5008;NS=2504   
              60684 60684 1              AC=62;AF=0.0123802;AN=5008;NS=2504   

                                        ac        af chromosome  
chromosome_id start end   alternate_id                           
9             60494 60494 1             27  0.005391         10  
              60515 60515 1              1  0.000200         10  
              60523 60523 1             91  0.018171         10  
              60607 60607 1              1  0.000200         10  
              60684 60684 1             62  0.012380         10  

[5 rows x 9 columns]

In [53]:
#str(VARIANT)
VARIANT.schema

u'<reference:string,alternate:string,id:string NULL DEFAULT null,qual:double NULL DEFAULT null,filter:string NULL DEFAULT null,info:string NULL DEFAULT null,ac:double NULL DEFAULT null,af:double NULL DEFAULT null,chromosome:string> [chromosome_id=0:*,1,0,start=0:*,10000000,0,end=0:*,10000000,0,alternate_id=0:19,20,0]'

In [55]:
#count(VARIANT)
VARIANT.count().toarray()[0][0]

19864.0

In [94]:
#GENOTYPE[9,60494,60494,1,0:4][]
GENOTYPE[9,60494,60494,1,0:5].todataframe()

,allele_1,allele_2,phase
sample_id,,,
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [73]:
#str(GENOTYPE)
GENOTYPE.schema

u'<allele_1:bool NULL DEFAULT null,allele_2:bool NULL DEFAULT null,phase:bool NULL DEFAULT null> [chromosome_id=0:*,1,0,start=0:*,10000000,0,end=0:*,10000000,0,alternate_id=0:19,20,0,sample_id=0:*,100,0]'

In [103]:
#count(GENOTYPE)
GENOTYPE.count().toarray()[0]

(49698274.0, 49698272.0, 49739456.0)

In [102]:
VARIANT[9,60494,60494,1:5].query

u'subarray(slice(py1101131311303_00012,chromosome_id,9,start,60494,end,60494),1,4)'

On the other hand, small arrays can easily be moved back and forth between SciDB and R, with one pitfall: SciDB arrays are zero-based, while R uses one-based indexing. But as genome scientists, we're right at home dealing with that!

## Transition/transversion ratio

The transition/transversion ratio (Ti/Tv) is a common quality metric for variant call sets. Let's compute Ti/Tv of all the variants with respect to the reference genome. This first calculation is on the variants only, not the individuals' genotypes, and thus involves only a modest amount of data.


In [211]:
# count biallelic SNPs
#SNP <- subset(VARIANT, "(reference='A' or reference='G' or reference='C' or reference='T') and
#                        (alternate='A' or alternate='G' or alternate='C' or alternate='T')")
SNP = sdb.afl.filter(VARIANT, "(reference='A' or reference='G' or reference='C' or reference='T') and \
                        (alternate='A' or alternate='G' or alternate='C' or alternate='T')")
#snps <- count(SNP)
snps = int(SNP.count().toarray()[0][0])
#snps
snps

18840

In [212]:
# annotate each SNP as to whether it's a transition (or else transversion)

#transitions_filter_str <- "(reference='A' and alternate='G') or (reference='G' and alternate='A') or
#                           (reference='C' and alternate='T') or (reference='T' and alternate='C')"
transitions_filter_str =   "(reference='A' and alternate='G') or (reference='G' and alternate='A') or \
                            (reference='C' and alternate='T') or (reference='T' and alternate='C')"

#SNP <- bind(SNP,"is_transition",paste("bool(iif(", transitions_filter_str, ",TRUE,FALSE))"))
SNP = sdb.afl.apply(SNP, "is_transition", "bool(iif(%s, 1, 0))" % transitions_filter_str)

#SNP <- scidbeval(SNP)
SNP = SNP.eval()

# count transitions

#ti <- count(SNP$is_transition %==% TRUE)
ti = int((sdb.afl.filter(SNP['is_transition'], "is_transition = TRUE").count()).toarray()[0])
ti

13209

In [213]:
# count transversions
#tv <- count(SNP$is_transition %==% FALSE)
tv = int((sdb.afl.filter(SNP['is_transition'], "is_transition = FALSE").count()).toarray()[0])
tv

5631

In [215]:
# report Ti/Tv
#stopifnot(ti+tv == snps)
assert ti + tv == snps
float(ti)/tv

2.345764517847629

Now let's look at the distribution of Ti/Tv across the individuals in the population. We use pipeR's `%>>%` operator to express a multi-step data processing pipeline, where `f(x) %>>% g(y)` denotes `g(f(x),y)`.

In [296]:
#calculate.titv <- function(G) {
#  (bind(G, "alt_copies", "iif(allele_1,1,0)+ iif(allele_2,1,0)")
#   %>>% merge(SNP$is_transition)
#   %>>% bind(c("ti", "tv"),
#             c("iif(is_transition, alt_copies, 0)",
#               "iif(not is_transition, alt_copies, 0)"))
#   %>>% aggregate(FUN="sum(ti) as ti, sum(tv) as tv", by = "sample_id")
#   %>>% scidbeval)
#}
def calculate_titv(G):
    expr = G.apply("alt_copies", "iif(allele_1,1,0)+ iif(allele_2,1,0)")
    expr = sdb.merge(expr, SNP['is_transition'])
    expr = expr.apply("ti", "iif(is_transition, alt_copies, 0)", \
                          "tv", "iif(not is_transition, alt_copies, 0)").\
            aggregate("sum(ti) as ti, sum(tv) as tv", "sample_id").\
            eval()
    return expr

#titv <- calculate.titv(GENOTYPE)[]
titv = calculate_titv(GENOTYPE).todataframe()

#invisible(hist(titv$ti/titv$tv, xlab="Ti/Tv", ylab="# individuals", main=""))
plt.hist(xx['ti']/xx['tv'])
plt.title("")
plt.xlabel("Ti/Tv")
plt.ylabel("# individuals")
plt.show()

All the data traversal is performed by SciDB in parallel; then the histogram buckets and counts are imported into R memory for plotting. Note that lazily-evaluated subexpressions can be stored, composed, and reused as R variables. This is often a lot nicer than formulating SQL!

We can also calculate Ti/Tv for just one individual, by filtering the input matrix:

In [312]:
#titv <- calculate.titv(merge(GENOTYPE, SAMPLE$sample %==% "HG03209"))[]
#titv$ti/titv$tv
temp = sdb.afl.filter(SAMPLE['sample'], 'sample = "HG03209"')
sdb.merge(GENOTYPE, temp).query


In [313]:
temp.query

u'filter(project(KG_SAMPLE,sample),sample = "HG03209")'

In [261]:
x.schema

u'<f0:double> [i0=0:2,1000,0,i1=0:3,1000,0]'